# Obter dados de CRIPTOMOEDAS

- Mudar moeda base para REAL
- Selecionar 300 itens por pagina
- Capturar dados por pagina
- Iterar sobre todas as demais paginas
- Salvar em arquivo

In [ ]:
from selenium import webdriver # Importa driver para navegadores pelo selenium
from selenium.webdriver.common.by import By # Importando classe By que server para selecionar os elementos
from selenium.webdriver.common.keys import Keys # Importo todas as teclas do teclado pelo pacote do Selenium
from selenium.webdriver.support.ui import Select # Import necessario para selecao de elementos em dropdown/selector
from selenium.webdriver.support.wait import WebDriverWait # Import da classe WAIT, serve para dizer o quanto se deve esperar pro navegador
from selenium.webdriver.support import expected_conditions as EC # Import de expected_conditions, e da nome reduzido (alias) EC
from dataclasses import dataclass # Import dataclass, como se fosse um objeto
import urllib.request # Import request para fazer uma requisicao para um determinado link e pegar todas as informacoes de determinado link
import os # Permite trabalhar com o Sistema Operacional
from selenium.webdriver.common.action_chains import ActionChains # Import da classe ActionChains
from pandas.io.clipboard import clipboard_set, clipboard_get # Import para mexer com menu de contexto
from io import StringIO # Import StringIO para PANDAS
import pandas as pd # Import do PANDAS
from selenium.webdriver.common import utils # Importa biblioteca com UTILIATARIAS para simplificar o codigo

driver = webdriver.Chrome() # Declara que driver é do chrome


# Iterando todas as paginas enquanto capturo todos os dados

In [37]:
with webdriver.Chrome() as driver: # Declaro que driver só existe enquanto faco as coisas nele
    driver.get('https://coingecko.com/pt') # Declaro pagina que estou acessando)
    driver.implicitly_wait(time_to_wait=30)
    driver.maximize_window()

    driver.execute_script('Modal.show("currency_selector")') # Abro modal de selecão de moeda
    driver.find_element(By.CSS_SELECTOR, 'div[data-iso-code="brl"]').click() # Seleciono moeda REAL

    driver.find_element(By.CSS_SELECTOR, 'div.gecko-pagination-selector button').click() # Abro dropdown para selecao de numero de itens por pagina
    driver.find_element(By.CSS_SELECTOR, 'span[data-url="/pt?items=300"]').click() # Seleciono que quero 300 itens por pagina

    dadosColetados = []

    while True:
        #Pegando dados da table
        df = pd.read_html(
            io = StringIO(driver.page_source),
            thousands = '.',
            decimal = ',',
            skiprows = 1
        )[0]

        #Renomeando colunas
        df.columns = ['Favorite', 'Id', 'Coin', 'Buy', 'Price', '1h', '24h', '7d', '30d', 'Volume in 24h', 'MarketCapitalization', 'FDV', 'FDVCapitalization', 'VariationGraph']
        
        #Adicionando a lista
        dadosColetados.append(df)

        try:
            #Indo para prox pag
            driver.find_element(By.CSS_SELECTOR, 'a[aria-label="next"]').click()
        except:
            print(f'Encerrando execucao na página {driver.current_url}')
            break

Encerrando execucao na página https://www.coingecko.com/pt?page=57&items=300


# Concatenando todas as paginas em uma crinado uma unica pagina com todas as Moedas

In [38]:
dfFinal = pd.concat(dadosColetados)

# Exportando dados

In [39]:
dfFinal.to_csv('criptoMoedas.csv', index=False) # Salvando em CSV

In [ ]:
# !pip install openpyxl # Biblioteca para engine para salvar em XLSX
dfFinal.to_excel('criptoMoedas.xlsx', index=False, engine='openpyxl') # Salvando em Excel

In [41]:
dfFinal.to_json('criptoMoedas.json', index=False, orient='records') # Salvando em Json

In [ ]:
import sqlite3 # Import banco de dados

In [43]:
with sqlite3.connect('database.db') as con: # Se existir um banco de dados com este nome ele se conecta / Se nao existir ele cria automaticamente
    dfFinal.to_sql(name='criptoMoedas', con=con, index=False, if_exists='replace') # Salvando em banco de dados